# Building And Testing A Complete Trading System

In [1]:
import yfinance as yf
import pandas_ta as pa
import plotly.graph_objects as go
import numpy as np

def get_data(symbol: str):
    data = yf.download(tickers=symbol, period='1000d', interval='1d')
    data.reset_index(inplace=True)
    return data
# Get the data
data = get_data('BTC-USD')

[*********************100%%**********************]  1 of 1 completed


## 1- Add rejection signal

In [2]:
def identify_rejection(data):
    # Create a new column for shooting star
    data['rejection'] = data.apply(
        lambda row: 2 if (
            ( (min(row['Open'], row['Close']) - row['Low']) > (1.5 * abs(row['Close'] - row['Open']))) and 
            (row['High'] - max(row['Close'], row['Open'])) < (0.8 * abs(row['Close'] - row['Open'])) and 
            (abs(row['Open'] - row['Close']) > row['Open'] * 0.001)
        ) else 1 if (
            (row['High'] - max(row['Open'], row['Close'])) > (1.5 * abs(row['Open'] - row['Close'])) and 
            (min(row['Close'], row['Open']) - row['Low']) < (0.8 * abs(row['Open'] - row['Close'])) and 
            (abs(row['Open'] - row['Close']) > row['Open'] * 0.001)
        ) else 0, axis=1
    )

    return data

data = identify_rejection(data)


In [3]:
data[data["rejection"]!=0]

,Date,Open,High,Low,Close,Adj Close,Volume,rejection
1,2021-09-08,46827.761719,47334.054688,44561.394531,46091.390625,46091.390625,49007762488,2
4,2021-09-11,44869.839844,45969.292969,44818.265625,45201.457031,45201.457031,34499835245,1
22,2021-09-29,41064.984375,42545.257812,40829.667969,41564.363281,41564.363281,30602359905,1
31,2021-10-08,53802.144531,55922.980469,53688.054688,53967.847656,53967.847656,34800873924,1
40,2021-10-17,60887.652344,61645.523438,59164.468750,61553.617188,61553.617188,29032367511,2
...,...,...,...,...,...,...,...,...
943,2024-04-07,68897.109375,70284.429688,68851.632812,69362.554688,69362.554688,21204930369,1
963,2024-04-27,63750.988281,63898.363281,62424.718750,63419.140625,63419.140625,19530783039,2
965,2024-04-29,63106.363281,64174.878906,61795.457031,63841.121094,63841.121094,26635912073,2
968,2024-05-02,58253.703125,59602.296875,56937.203125,59123.433594,59123.433594,32711813559,2


In [4]:
def pointpos(x, xsignal):
    if x[xsignal]==1:
        return x['High']+1e-4
    elif x[xsignal]==2:
        return x['Low']-1e-4
    else:
        return np.nan

def plot_with_signal(dfpl):

    fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                    open=dfpl['Open'],
                    high=dfpl['High'],
                    low=dfpl['Low'],
                    close=dfpl['Close'])])

    fig.update_layout(
        autosize=False,
        width=1000,
        height=800, 
        paper_bgcolor='black',
        plot_bgcolor='black')
    fig.update_xaxes(gridcolor='black')
    fig.update_yaxes(gridcolor='black')
    fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                    marker=dict(size=8, color="MediumPurple"),
                    name="Signal")
    fig.show()

data['pointpos'] = data.apply(lambda row: pointpos(row,"rejection"), axis=1)
plot_with_signal(data[10:110])

## 2- Support and Resistance FUNCTIONS

In [5]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    if ( df1.Low[l-n1:l].min() < df1.Low[l] or
        df1.Low[l+1:l+n2+1].min() < df1.Low[l] ):
        return 0
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    if ( df1.High[l-n1:l].max() > df1.High[l] or
       df1.High[l+1:l+n2+1].max() > df1.High[l] ):
        return 0
    return 1

## 3- Close to resistance and support testing

In [6]:
def closeResistance(l, levels, lim, df):
    if len(levels) == 0:
        return 0
    c1 = abs(df['High'][l] - min(levels, key=lambda x: abs(x - df['High'][l]))) <= lim
    c2 = abs(max(df['Open'][l], df['Close'][l]) - min(levels, key=lambda x: abs(x - df['High'][l]))) <= lim
    c3 = min(df['Open'][l], df['Close'][l]) < min(levels, key=lambda x: abs(x - df['High'][l]))
    c4 = df['Low'][l] < min(levels, key=lambda x: abs(x - df['High'][l]))
    if (c1 or c2) and c3 and c4:
        return min(levels, key=lambda x: abs(x - df['High'][l]))
    else:
        return 0

def closeSupport(l, levels, lim, df):
    if len(levels) == 0:
        return 0
    c1 = abs(df['Low'][l] - min(levels, key=lambda x: abs(x - df['Low'][l]))) <= lim
    c2 = abs(min(df['Open'][l], df['Close'][l]) - min(levels, key=lambda x: abs(x - df['Low'][l]))) <= lim
    c3 = max(df['Open'][l], df['Close'][l]) > min(levels, key=lambda x: abs(x - df['Low'][l]))
    c4 = df['High'][l] > min(levels, key=lambda x: abs(x - df['Low'][l]))
    if (c1 or c2) and c3 and c4:
        return min(levels, key=lambda x: abs(x - df['Low'][l]))
    else:
        return 0

def is_below_resistance(l, level_backCandles, level, df):
    return df.loc[l-level_backCandles:l-1, 'High'].max() < level

def is_above_support(l, level_backCandles, level, df):
    return df.loc[l-level_backCandles:l-1, 'Low'].min() > level

In [7]:
def check_candle_signal(l, n1, n2, levelbackCandles, windowbackCandles, df):
    ss = []
    rr = []
    for subrow in range(l-levelbackCandles, l-n2+1):
        if support(df, subrow, n1, n2):
            ss.append(df.Low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.High[subrow])

    ss.sort() #keep lowest support when popping a level
    for i in range(1,len(ss)):
        if(i>=len(ss)):
            break
        if abs(ss[i]-ss[i-1])/ss[i]<=0.001: # merging close distance levels
            ss.pop(i)

    rr.sort(reverse=True) # keep highest resistance when popping one
    for i in range(1,len(rr)):
        if(i>=len(rr)):
            break
        if abs(rr[i]-rr[i-1])/rr[i]<=0.001: # merging close distance levels
            rr.pop(i)

    #----------------------------------------------------------------------
    # joined levels
    rrss = rr+ss
    rrss.sort()
    for i in range(1,len(rrss)):
        if(i>=len(rrss)):
            break
        if abs(rrss[i]-rrss[i-1])/rrss[i]<=0.001: # merging close distance levels
            rrss.pop(i)
    cR = closeResistance(l, rrss, df.Close[l]*0.003, df)
    cS = closeSupport(l, rrss, df.Close[l]*0.003, df)
    #----------------------------------------------------------------------

    # cR = closeResistance(l, rr, 150e-5, df)
    # cS = closeSupport(l, ss, 150e-5, df)
    #print(rrss,df.Close*0.002)
    if (df.rejection[l] == 1 and cR and is_below_resistance(l,windowbackCandles,cR, df)):
        return 1
    elif(df.rejection[l] == 2 and cS and is_above_support(l,windowbackCandles,cS, df)):
        return 2
    else:
        return 0

In [8]:
from tqdm import tqdm

n1 = 8
n2 = 8
levelbackCandles = 60
windowbackCandles = n2

signal = [0 for i in range(len(data))]

for row in tqdm(range(levelbackCandles+n1, len(data)-n2)):
    signal[row] = check_candle_signal(row, n1, n2, levelbackCandles, windowbackCandles, data)

data["signal"] = signal

100%|██████████| 924/924 [00:10<00:00, 89.74it/s] 


In [9]:
data[data["signal"]!=0]

,Date,Open,High,Low,Close,Adj Close,Volume,rejection,pointpos,signal
156,2022-02-10,44347.800781,45661.171875,43402.808594,43565.113281,43565.113281,32142048537,1,45661.171975,1
199,2022-03-25,43964.546875,44999.492188,43706.285156,44348.730469,44348.730469,30574413034,1,44999.492287,1
401,2022-10-13,19156.966797,19453.328125,18319.822266,19382.904297,19382.904297,44219840004,2,18319.822166,2
479,2022-12-30,16641.330078,16643.427734,16408.474609,16602.585938,16602.585938,15929162910,2,16408.474509,2
625,2023-05-25,26329.460938,26591.519531,25890.593750,26476.207031,26476.207031,13851122697,2,25890.593650,2
667,2023-07-06,30507.150391,31460.052734,29892.226562,29909.337891,29909.337891,21129219509,1,31460.052834,1


In [10]:
data['pointpos'] = data.apply(lambda row: pointpos(row,"signal"), axis=1)
plot_with_signal(data[500:750])

## 4- Backtesting

In [11]:
data.set_index("Date", inplace=True)

In [12]:
data

,Open,High,Low,Close,Adj Close,Volume,rejection,pointpos,signal
Date,,,,,,,,,
2021-09-07,52660.480469,52853.765625,43285.207031,46811.128906,46811.128906,65210059683,0,NaN,0
2021-09-08,46827.761719,47334.054688,44561.394531,46091.390625,46091.390625,49007762488,2,NaN,0
2021-09-09,45774.742188,47261.949219,45669.738281,46391.421875,46391.421875,38672657013,0,NaN,0
2021-09-10,46396.664062,47031.742188,44344.484375,44883.910156,44883.910156,39154666597,0,NaN,0
2021-09-11,44869.839844,45969.292969,44818.265625,45201.457031,45201.457031,34499835245,1,NaN,0
...,...,...,...,...,...,...,...,...,...
2024-05-29,68296.351562,68852.460938,67101.492188,67578.093750,67578.093750,26707072906,0,NaN,0
2024-05-30,67576.085938,69500.539062,67118.078125,68364.992188,68364.992188,29509712534,0,NaN,0
2024-05-31,68362.515625,68999.562500,66633.421875,67491.414062,67491.414062,27387283769,0,NaN,0


In [13]:
data['ATR'] = pa.atr(high=data.High, low=data.Low, close=data.Close, length=14)
data['RSI'] = pa.rsi(data.Close, length=5)

In [14]:
def SIGNAL():
    return data.signal

In [15]:
#A new strategy needs to extend Strategy class and override its two abstract methods: init() and next().
#Method init() is invoked before the strategy is run. Within it, one ideally precomputes in efficient, 
#vectorized manner whatever indicators and signals the strategy depends on.
#Method next() is then iteratively called by the Backtest instance, once for each data point (data frame row), 
#simulating the incremental availability of each new full candlestick bar.

#Note, backtesting.py cannot make decisions / trades within candlesticks — any new orders are executed on the
#next candle's open (or the current candle's close if trade_on_close=True). 
#If you find yourself wishing to trade within candlesticks (e.g. daytrading), you instead need to begin 
#with more fine-grained (e.g. hourly) data.

### 4.1- Using Fixed SL and TP rules

In [16]:
# Trader fixed SL and TP
from backtesting import Strategy, Backtest
class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        self.ratio = 2
        self.risk_perc = 0.1

    def next(self):
        super().next() 
        if self.signal1==2:
            sl1 = self.data.Close[-1] - self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] + (self.data.Close[-1]*self.risk_perc)*self.ratio
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] - (self.data.Close[-1]*self.risk_perc)*self.ratio
            self.sell(sl=sl1, tp=tp1)
bt = Backtest(data, MyCandlesStrat, cash=100_000, commission=.02)
stat = bt.run()
stat

Start                     2021-09-07 00:00:00
End                       2024-06-02 00:00:00
Duration                    999 days 00:00:00
Exposure Time [%]                        14.7
Equity Final [$]                159472.617773
Equity Peak [$]                 159472.617773
Return [%]                          59.472618
Buy & Hold Return [%]               45.038143
Return (Ann.) [%]                   18.571533
Volatility (Ann.) [%]               19.900528
Sharpe Ratio                         0.933218
Sortino Ratio                        1.839677
Calmar Ratio                         1.030729
Max. Drawdown [%]                  -18.017854
Avg. Drawdown [%]                   -3.831925
Max. Drawdown Duration      204 days 00:00:00
Avg. Drawdown Duration       18 days 00:00:00
# Trades                                    5
Win Rate [%]                             80.0
Best Trade [%]                      18.378618
Worst Trade [%]                    -11.763017
Avg. Trade [%]                    

In [17]:
bt.plot()

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.activ

GridPlot(id='p1351', ...)

### 4.2- Using the RSI for Exit Signals

In [18]:
from backtesting import Strategy, Backtest

class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        self.ratio = 2
        self.risk_perc = 0.1

    def next(self):
        super().next()
        
        if len(self.trades)>0:
            if self.trades[-1].is_long and self.data.RSI[-1]>=80:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=20:
                self.trades[-1].close()

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] + (self.data.Close[-1]*self.risk_perc)*self.ratio
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.Close[-1] + self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] - (self.data.Close[-1]*self.risk_perc)*self.ratio
            self.sell(sl=sl1, tp=tp1)
bt = Backtest(data, MyCandlesStrat, cash=100_000, commission=.02)
stat = bt.run()
stat

Start                     2021-09-07 00:00:00
End                       2024-06-02 00:00:00
Duration                    999 days 00:00:00
Exposure Time [%]                        12.4
Equity Final [$]                142169.216602
Equity Peak [$]                 142169.216602
Return [%]                          42.169217
Buy & Hold Return [%]               45.038143
Return (Ann.) [%]                   13.703552
Volatility (Ann.) [%]               14.473342
Sharpe Ratio                         0.946813
Sortino Ratio                        1.964504
Calmar Ratio                         1.510732
Max. Drawdown [%]                   -9.070801
Avg. Drawdown [%]                   -3.050988
Max. Drawdown Duration      197 days 00:00:00
Avg. Drawdown Duration       24 days 00:00:00
# Trades                                    6
Win Rate [%]                            100.0
Best Trade [%]                      13.187296
Worst Trade [%]                      0.934384
Avg. Trade [%]                    

In [19]:
bt.plot()

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.activ

GridPlot(id='p1730', ...)

### 4.3- ATR based SL and TP

In [20]:
# ATR related SL and TP
from backtesting import Strategy, Backtest
import numpy as np

class MyCandlesStrat(Strategy): 
    atr_f = 0.5
    ratio_f = 1.5
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1==2:
            sl1 = self.data.Close[-1] - self.data.ATR[-1]/self.atr_f
            tp1 = self.data.Close[-1] + self.data.ATR[-1]*self.ratio_f/self.atr_f
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + self.data.ATR[-1]/self.atr_f
            tp1 = self.data.Close[-1] - self.data.ATR[-1]*self.ratio_f/self.atr_f
            self.sell(sl=sl1, tp=tp1)
bt = Backtest(data, MyCandlesStrat, cash=100_000, commission=.000)
stat = bt.run()
stat

Start                     2021-09-07 00:00:00
End                       2024-06-02 00:00:00
Duration                    999 days 00:00:00
Exposure Time [%]                         7.7
Equity Final [$]                108280.281047
Equity Peak [$]                 126416.191336
Return [%]                           8.280281
Buy & Hold Return [%]               45.038143
Return (Ann.) [%]                    2.946248
Volatility (Ann.) [%]               10.358483
Sharpe Ratio                         0.284429
Sortino Ratio                        0.467996
Calmar Ratio                         0.205368
Max. Drawdown [%]                  -14.346193
Avg. Drawdown [%]                   -3.574704
Max. Drawdown Duration      371 days 00:00:00
Avg. Drawdown Duration       68 days 00:00:00
# Trades                                    6
Win Rate [%]                             50.0
Best Trade [%]                      13.404652
Worst Trade [%]                     -7.526472
Avg. Trade [%]                    

In [21]:
bt.plot()

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.activ

GridPlot(id='p2109', ...)

### 4.4- Trail Stop

In [22]:
#fixed distance Trailing SL
from backtesting import Strategy, Backtest

class MyCandlesStrat(Strategy):
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        sltr=self.data.Close[-1]*0.02

        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - sltr)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + sltr) 
        
        if self.signal1==2 and len(self.trades)==0: 
            sl1 = self.data.Close[-1] - sltr
            self.buy(sl=sl1)
        elif self.signal1==1 and len(self.trades)==0: 
            sl1 = self.data.Close[-1] + sltr
            self.sell(sl=sl1)


bt = Backtest(data, MyCandlesStrat, cash=100_000, commission=.000)
stat = bt.run()
stat

Start                     2021-09-07 00:00:00
End                       2024-06-02 00:00:00
Duration                    999 days 00:00:00
Exposure Time [%]                         4.1
Equity Final [$]                120687.414688
Equity Peak [$]                 125332.372813
Return [%]                          20.687415
Buy & Hold Return [%]               45.038143
Return (Ann.) [%]                     7.10423
Volatility (Ann.) [%]                 7.02161
Sharpe Ratio                         1.011767
Sortino Ratio                        2.877389
Calmar Ratio                         1.449996
Max. Drawdown [%]                   -4.899484
Avg. Drawdown [%]                   -2.688224
Max. Drawdown Duration      371 days 00:00:00
Avg. Drawdown Duration      210 days 00:00:00
# Trades                                    6
Win Rate [%]                             50.0
Best Trade [%]                      24.949697
Worst Trade [%]                     -2.004569
Avg. Trade [%]                    

In [23]:
bt.plot()

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.activ

GridPlot(id='p2488', ...)

In [24]:
#ATR based Trailing Stop
from backtesting import Strategy, Backtest

class MyCandlesStrat(Strategy):
    atr_f = 0.6
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        self.sltr=0

    def next(self):
        super().next()
        
        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - self.sltr)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + self.sltr)

        if self.signal1==2 and len(self.trades)==0: 
            self.sltr=self.data.ATR[-1]/self.atr_f
            sl1 = self.data.Close[-1] - self.data.ATR[-1]/self.atr_f
            self.buy(sl=sl1)
        elif self.signal1==1 and len(self.trades)==0: 
            self.sltr=self.data.ATR[-1]/self.atr_f
            sl1 = self.data.Close[-1] + self.data.ATR[-1]/self.atr_f
            self.sell(sl=sl1)
bt = Backtest(data, MyCandlesStrat, cash=100_000, commission=.000)
stat = bt.run()
stat

Start                     2021-09-07 00:00:00
End                       2024-06-02 00:00:00
Duration                    999 days 00:00:00
Exposure Time [%]                         8.0
Equity Final [$]                 124734.71602
Equity Peak [$]                 135971.769236
Return [%]                          24.734716
Buy & Hold Return [%]               45.038143
Return (Ann.) [%]                    8.401522
Volatility (Ann.) [%]               12.374055
Sharpe Ratio                         0.678963
Sortino Ratio                        1.315312
Calmar Ratio                         0.693442
Max. Drawdown [%]                  -12.115681
Avg. Drawdown [%]                   -4.653051
Max. Drawdown Duration      371 days 00:00:00
Avg. Drawdown Duration      140 days 00:00:00
# Trades                                    6
Win Rate [%]                        66.666667
Best Trade [%]                      24.433671
Worst Trade [%]                     -6.271636
Avg. Trade [%]                    

## 5- Lot sizing and trade management

In [25]:
class MyCandlesStrat(Strategy):
    lotsize = 1 
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        self.ratio = 1.
        self.risk_perc = 0.1

    def next(self):
        super().next() 
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] + (self.data.Close[-1]*self.risk_perc)*self.ratio*0.8
            tp2 = self.data.Close[-1] + (self.data.Close[-1]*self.risk_perc)*self.ratio*1.2
            self.buy(sl=sl1, tp=tp1, size=self.lotsize)
            self.buy(sl=sl1, tp=tp2, size=self.lotsize)
        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.Close[-1] + self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] - (self.data.Close[-1]*self.risk_perc)*self.ratio*0.8
            tp2 = self.data.Close[-1] - (self.data.Close[-1]*self.risk_perc)*self.ratio*1.2
            self.sell(sl=sl1, tp=tp1, size=self.lotsize)
            self.sell(sl=sl1, tp=tp2, size=self.lotsize)
bt = Backtest(data, MyCandlesStrat, cash=100_000, margin=1/1, commission=.02)
stat = bt.run()
stat

Start                     2021-09-07 00:00:00
End                       2024-06-02 00:00:00
Duration                    999 days 00:00:00
Exposure Time [%]                        14.3
Equity Final [$]                124595.388398
Equity Peak [$]                 124595.388398
Return [%]                          24.595388
Buy & Hold Return [%]               45.038143
Return (Ann.) [%]                    8.357311
Volatility (Ann.) [%]               10.075229
Sharpe Ratio                         0.829491
Sortino Ratio                        1.599057
Calmar Ratio                         1.116804
Max. Drawdown [%]                   -7.483241
Avg. Drawdown [%]                   -2.139931
Max. Drawdown Duration      204 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                   12
Win Rate [%]                        91.666667
Best Trade [%]                      10.216479
Worst Trade [%]                    -11.763017
Avg. Trade [%]                    